In [12]:
# IMPORTS
import numpy as np
import pandas as pd
import os
import geoip2.database
import geoip2.errors
import geopy
from typing import Tuple

In [13]:
# CONSTS
GEOIP_PATH = "db/GeoLite2-City_20240109/GeoLite2-City.mmdb"
SCAN_PATH = "csvs/scan_2024-01-23_15-11.csv.gz"
ENR_SCAN_PATH = "csvs/enriched_scan.csv.gz"
PLOT_PATH = "plots"
# INIT
os.makedirs(PLOT_PATH, exist_ok=True)
import df_logic
df_logic.init_geo(GEOIP_PATH)
import plots

In [11]:
# call to create enriched dataframe
df_logic.create_enriched_data(SCAN_PATH, ENR_SCAN_PATH, truncate=False)

truncating enrichted CSV file
now creating enriched file in chunks
chunk 0 completed
chunk 1 completed
chunk 2 completed
chunk 3 completed
chunk 4 completed
chunk 5 completed


In [ ]:
# tests
geoip2.database.Reader("")